In [1]:
!pip install flask pyngrok openai google-generativeai scikit-learn joblib APScheduler requests beautifulsoup4 python-dotenv --quiet

In [2]:
readme_content = """
# ... (cole o conteúdo do README.md aqui) ...
"""
with open("README.md", "w", encoding="utf-8") as f:
    f.write(readme_content)
print("Arquivo README.md salvo.")

Arquivo README.md salvo.


In [3]:
requirements_content = """
# ... (cole o conteúdo do requirements.txt aqui) ...
"""
with open("requirements.txt", "w", encoding="utf-8") as f:
    f.write(requirements_content)
print("Arquivo requirements.txt salvo.")

Arquivo requirements.txt salvo.


In [4]:
!pip install flask pyngrok openai scikit-learn joblib APScheduler requests beautifulsoup4 python-dotenv --quiet

In [5]:
!pip install flask pyngrok openai google-generativeai scikit-learn joblib APScheduler requests beautifulsoup4 python-dotenv numpy --quiet

In [6]:
import os
if not os.path.exists("templates"):
    os.makedirs("templates")

app_py_content = """
# Imports necessários
from flask import Flask, request, render_template, jsonify
# ... COLE TODO O CÓDIGO PYTHON ATUALIZADO AQUI ...
default_port = int(os.getenv("PORT", 5005))
try_run_app_with_ngrok(initial_port=default_port)
"""
with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_py_content)
print("Arquivo app.py salvo com a correção do ngrok.connect().")

Arquivo app.py salvo com a correção do ngrok.connect().


In [7]:
!python app.py

Traceback (most recent call last):
  File "/content/app.py", line 5, in <module>
    default_port = int(os.getenv("PORT", 5005))
                       ^^
NameError: name 'os' is not defined


In [8]:
import os
if not os.path.exists("templates"):
    os.makedirs("templates")

app_py_content = """
# Imports necessários
from flask import Flask, request, render_template, jsonify
# ... COLE TODO O CÓDIGO PYTHON ATUALIZADO AQUI ...
default_port = int(os.getenv("PORT", 5005))
try_run_app_with_ngrok(initial_port=default_port)
"""
with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_py_content)
print("Arquivo app.py salvo com lógica Multi-IA.")

Arquivo app.py salvo com lógica Multi-IA.


In [ ]:
# Imports necessários
from flask import Flask, request, render_template, jsonify
import openai # Para ChatGPT
import google.generativeai as genai # Para Gemini
import json
import os
import hashlib
import logging
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np # Para cálculos numéricos, especialmente com scikit-learn
from threading import Thread
from apscheduler.schedulers.background import BackgroundScheduler
import requests
from bs4 import BeautifulSoup
import time
from dotenv import load_dotenv
import socket
from pyngrok import ngrok, conf
from functools import wraps # Para o decorador de autenticação

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Inicialização do Flask app
app = Flask(__name__)

# --- Configuração das Chaves de API e Outras Configurações ---
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
RETRAIN_API_KEY_SECRET = os.getenv("RETRAIN_API_KEY") # Chave para proteger o endpoint de retreinamento

# Configuração do cliente OpenAI
if OPENAI_API_KEY:
    if OPENAI_API_KEY.startswith("sk-"):
        openai.api_key = OPENAI_API_KEY
        logging.info("Chave da API OpenAI carregada e parece válida.")
    else:
        logging.error("Chave da API OpenAI (OPENAI_API_KEY) fornecida não parece ser válida (não começa com 'sk-'). Funcionalidade do ChatGPT desabilitada.")
        OPENAI_API_KEY = None # Desabilita se a chave for inválida
else:
    logging.warning("Chave da API OpenAI (OPENAI_API_KEY) não encontrada. Funcionalidade do ChatGPT desabilitada.")

# Configuração do cliente Gemini
if GEMINI_API_KEY:
    try:
        # Chaves Gemini podem começar com 'AIza...'
        genai.configure(api_key=GEMINI_API_KEY)
        logging.info("Chave da API Gemini configurada com sucesso.")
    except Exception as e:
        logging.error(f"Erro ao configurar a API Gemini: {e}. Funcionalidade do Gemini desabilitada.")
        GEMINI_API_KEY = None
else:
    logging.warning("Chave da API Gemini (GEMINI_API_KEY) não encontrada. Funcionalidade do Gemini desabilitada.")

# Configuração do logging
logging.basicConfig(
    filename="log_app.log",
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(module)s - %(funcName)s - %(message)s'
)
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO) # Mostra logs INFO e acima no console
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
# Evita adicionar handlers duplicados se a célula/script for executado múltiplas vezes
logger = logging.getLogger()
if not any(isinstance(h, logging.StreamHandler) for h in logger.handlers):
    logger.addHandler(console_handler)
logger.setLevel(logging.INFO) # Garante que o logger raiz também tem o nível INFO


# Constantes para nomes de arquivos e configurações
EXEMPLOS_GOLPES_FILE = "exemplos_golpes.json"
MODELO_PKL_FILE = "modelo_golpe_guard.pkl"
MODEL_EVALUATION_FILE = "model_evaluation_report.txt"
MODELO_LOCAL_CONFIANCA_LIMIAR = 0.70 # Limiar de confiança para o modelo local
API_RETRY_ATTEMPTS = 2 # Número de tentativas para chamadas de API externas
API_RETRY_DELAY = 3 # Delay em segundos entre tentativas

# ---------- Decorador de Autenticação para Rotas Protegidas ------------
def require_api_key(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        if not RETRAIN_API_KEY_SECRET:
            logging.error("Chave RETRAIN_API_KEY não configurada no servidor. Endpoint desabilitado.")
            return jsonify({"erro": "Endpoint de retreinamento não configurado corretamente no servidor."}), 500

        provided_token = request.headers.get('X-Retrain-Token')
        if provided_token and provided_token == RETRAIN_API_KEY_SECRET:
            return f(*args, **kwargs)
        else:
            logging.warning("Tentativa de acesso não autorizado ao endpoint de retreinamento.")
            return jsonify({"erro": "Não autorizado. Token de retreinamento inválido ou não fornecido."}), 401
    return decorated_function

# ---------- Funções base para manipular a base de dados local ------------
def carregar_base():
    try:
        with open(EXEMPLOS_GOLPES_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        return []
    except json.JSONDecodeError:
        logging.error(f"Erro ao decodificar JSON do arquivo {EXEMPLOS_GOLPES_FILE}.")
        return []

def salvar_base(dados):
    try:
        with open(EXEMPLOS_GOLPES_FILE, "w", encoding="utf-8") as f:
            json.dump(dados, f, indent=4, ensure_ascii=False)
    except IOError as e:
        logging.error(f"Erro de I/O ao salvar {EXEMPLOS_GOLPES_FILE}: {e}")

def adicionar_nova_entrada(base_dados, mensagem, classificacao):
    # Normaliza a classificação para garantir consistência
    classificacao_lower = str(classificacao).lower() # Garante que é string antes de lower()
    if "não é golpe" in classificacao_lower:
        classificacao_normalizada = "Não é golpe"
    elif "golpe" in classificacao_lower:
        classificacao_normalizada = "Golpe"
    else:
        logging.warning(f"Classificação '{classificacao}' não reconhecida para adicionar à base. Mensagem: {mensagem[:50]}...")
        return base_dados

    hash_msg_nova = hashlib.sha256(mensagem.encode("utf-8")).hexdigest()
    mensagem_encontrada = False
    for entrada in base_dados:
        hash_msg_existente = hashlib.sha256(entrada["mensagem"].encode("utf-8")).hexdigest()
        if hash_msg_existente == hash_msg_nova:
            if entrada["classificacao"] != classificacao_normalizada:
                logging.info(f"Atualizando classificação da mensagem (hash: {hash_msg_nova[:7]}...) de '{entrada['classificacao']}' para '{classificacao_normalizada}'.")
                entrada["classificacao"] = classificacao_normalizada
            else: # Adicionado log para quando a classificação é a mesma
                 logging.info(f"Mensagem (hash: {hash_msg_nova[:7]}...) já existe com a mesma classificação '{classificacao_normalizada}'. Nenhuma alteração na base.")
            mensagem_encontrada = True
            break
    if not mensagem_encontrada:
        base_dados.append({"mensagem": mensagem, "classificacao": classificacao_normalizada})
        logging.info(f"Nova entrada adicionada à base (hash: {hash_msg_nova[:7]}...): {classificacao_normalizada}")
    return base_dados

# ---------- Treinamento, Avaliação e Carregamento do Modelo Local ------------
def avaliar_modelo(modelo, X_test, y_test):
    if not hasattr(modelo, "predict"):
        logging.warning("Tentativa de avaliar um modelo inválido ou não treinado.")
        return "Modelo inválido ou não treinado para avaliação."
    try:
        y_pred = modelo.predict(X_test)
        labels = sorted(list(set(y_test) | set(y_pred)))
        if not labels: labels = None

        report = classification_report(y_test, y_pred, zero_division=0, labels=labels)
        cm = confusion_matrix(y_test, y_pred, labels=labels)
        accuracy = accuracy_score(y_test, y_pred)

        evaluation_text = f"Relatório de Avaliação do Modelo Local ({time.strftime('%Y-%m-%d %H:%M:%S')}):\n"
        evaluation_text += f"Acurácia: {accuracy:.4f}\n"
        evaluation_text += "Relatório de Classificação:\n"
        evaluation_text += report + "\n"
        evaluation_text += "Matriz de Confusão:\n"
        evaluation_text += str(cm) + "\n"
        evaluation_text += "------------------------------------------------------\n"

        logging.info(f"Avaliação do modelo local: Acurácia={accuracy:.4f}")
        with open(MODEL_EVALUATION_FILE, "a", encoding="utf-8") as f:
            f.write(evaluation_text)
        return evaluation_text
    except Exception as e:
        logging.error(f"Erro durante a avaliação do modelo: {e}")
        return f"Erro durante a avaliação do modelo: {e}"

def treinar_modelo():
    dados = carregar_base()
    if not dados:
        logging.warning("Base de dados vazia. Pulando treinamento.")
        return None

    mensagens = [d["mensagem"] for d in dados if isinstance(d.get("mensagem"), str) and d.get("mensagem") and d.get("classificacao") in ["Golpe", "Não é golpe"]]
    etiquetas = [d["classificacao"] for d in dados if isinstance(d.get("mensagem"), str) and d.get("mensagem") and d.get("classificacao") in ["Golpe", "Não é golpe"]]

    if len(mensagens) < 5 or len(set(etiquetas)) < 2:
        logging.warning(f"Dados insuficientes para treinamento (mensagens válidas: {len(mensagens)}, classes distintas: {len(set(etiquetas))}). Pulando.")
        return None

    modelo = make_pipeline(TfidfVectorizer(), MultinomialNB())

    try:
        if len(set(etiquetas)) > 1 and len(mensagens) > 1 :
            min_samples_per_class = min(np.unique(etiquetas, return_counts=True)[1]) if etiquetas else 0
            test_size = 0.2
            # Ajusta test_size ou desabilita stratify se houver poucas amostras
            can_stratify = min_samples_per_class >= 2
            actual_test_size = test_size if len(mensagens) * test_size >= 1 else 0.1 # Garante pelo menos 1 no teste se possível
            if len(mensagens) * actual_test_size < 1 : # Se ainda não for possível ter amostra de teste
                X_train, X_test, y_train, y_test = mensagens, [], etiquetas, []
            elif can_stratify:
                 X_train, X_test, y_train, y_test = train_test_split(mensagens, etiquetas, test_size=actual_test_size, random_state=42, stratify=etiquetas)
            else:
                 logging.warning(f"Não é possível estratificar devido a poucas amostras na menor classe ({min_samples_per_class}). Usando split simples.")
                 X_train, X_test, y_train, y_test = train_test_split(mensagens, etiquetas, test_size=actual_test_size, random_state=42)
        else:
            X_train, X_test, y_train, y_test = mensagens, [], etiquetas, []

        if not X_train:
            logging.warning("Conjunto de treino vazio. Pulando treinamento.")
            return None

        modelo.fit(X_train, y_train)
        logging.info("Modelo local treinado com sucesso.")
        joblib.dump(modelo, MODELO_PKL_FILE)
        logging.info(f"Modelo local salvo em {MODELO_PKL_FILE}.")

        if X_test and y_test :
            avaliar_modelo(modelo, X_test, y_test)
        else:
            logging.warning("Conjunto de teste vazio. Avaliação do modelo não realizada.")

        return modelo
    except ValueError as ve:
        logging.error(f"Erro de valor durante o treinamento: {ve}")
        return None
    except Exception as e:
        logging.error(f"Falha ao treinar ou salvar o modelo local: {e}")
        return None

modelo_treinado_global = None
try:
    if os.path.exists(MODELO_PKL_FILE):
        modelo_treinado_global = joblib.load(MODELO_PKL_FILE)
        logging.info(f"Modelo local carregado de {MODELO_PKL_FILE}.")
    else:
        logging.info(f"Arquivo de modelo {MODELO_PKL_FILE} não encontrado. Tentando treinar novo modelo.")
        modelo_treinado_global = treinar_modelo()
except Exception as e:
    logging.error(f"Erro ao carregar modelo salvo de {MODELO_PKL_FILE}: {e}. Tentando treinar novo modelo.")
    modelo_treinado_global = treinar_modelo()

# ---------- Funções de Classificação com diferentes IAs e Retries ------------
def api_call_with_retry(api_function, *args, **kwargs):
    for attempt in range(API_RETRY_ATTEMPTS):
        try:
            return api_function(*args, **kwargs)
        except (openai.error.RateLimitError, openai.error.APIError, openai.error.ServiceUnavailableError, requests.exceptions.RequestException, genai.types.generation_types.StopCandidateException) as e:
            logging.warning(f"Erro de API na tentativa {attempt + 1}/{API_RETRY_ATTEMPTS} para {api_function.__name__}: {e}")
            if attempt + 1 == API_RETRY_ATTEMPTS:
                logging.error(f"Todas as {API_RETRY_ATTEMPTS} tentativas falharam para {api_function.__name__}.")
                raise
            time.sleep(API_RETRY_DELAY * (attempt + 1))
        except openai.error.AuthenticationError as auth_e:
            logging.error(f"Erro de autenticação com {api_function.__name__}: {auth_e}")
            raise
        except Exception as general_exception:
            if "API_KEY_INVALID" in str(general_exception) or "PERMISSION_DENIED" in str(general_exception) or ("google.api_core.exceptions.PermissionDenied" in str(type(general_exception))):
                logging.error(f"Erro de autenticação com {api_function.__name__} (possivelmente Gemini): {general_exception}")
                raise general_exception

            logging.warning(f"Erro de API (não autenticação) na tentativa {attempt + 1}/{API_RETRY_ATTEMPTS} para {api_function.__name__}: {general_exception}")
            if attempt + 1 == API_RETRY_ATTEMPTS:
                raise
            time.sleep(API_RETRY_DELAY * (attempt + 1))
    return None

def construir_prompt_comum(mensagem, exemplos_base):
    prompt_exemplos_str = "".join([f'Mensagem: "{ex["mensagem"]}"\nClassificação: {ex["classificacao"]}\n\n' for ex in exemplos_base[-10:]])
    return (
        "Você é um assistente especializado em detetar fraudes e golpes em mensagens de texto. "
        "Sua tarefa é classificar a mensagem fornecida pelo utilizador como 'Golpe' ou 'Não é golpe'. "
        "Analise cuidadosamente o conteúdo da mensagem. Considere os seguintes exemplos:\n\n"
        f"{prompt_exemplos_str}"
        f"Agora, classifique esta mensagem: \"{mensagem}\"\nResponda APENAS com 'Golpe' ou 'Não é golpe'.\nClassificação:"
    )

def _openai_api_call(model, messages, temperature, max_tokens):
    return openai.ChatCompletion.create(model=model, messages=messages, temperature=temperature, max_tokens=max_tokens)

def classificar_com_openai(mensagem, exemplos_base):
    if not OPENAI_API_KEY:
        logging.warning("OpenAI API Key não configurada.")
        return None, "OpenAI (Não Configurado)"
    prompt_completo = construir_prompt_comum(mensagem, exemplos_base)
    messages = [
        {"role": "system", "content": "Você é um assistente que deteta se mensagens são golpes ou não. Responda APENAS com 'Golpe' ou 'Não é golpe'."},
        {"role": "user", "content": prompt_completo}
    ]
    try:
        logging.info(f"Enviando para OpenAI: '{mensagem[:30]}...'")
        resposta_obj = api_call_with_retry(_openai_api_call, model="gpt-4-turbo", messages=messages, temperature=0, max_tokens=10)
        if resposta_obj:
            resultado_ia = resposta_obj.choices[0].message.content.strip().lower()
            if resultado_ia in ["golpe", "não é golpe"]:
                logging.info(f"OpenAI classificou como: {resultado_ia}")
                return resultado_ia, "OpenAI (GPT-4 Turbo)"
            logging.warning(f"Resposta inesperada da OpenAI: '{resultado_ia}'.")
        return None, "OpenAI (GPT-4 Turbo)"
    except openai.error.AuthenticationError as e:
        logging.error(f"Erro de autenticação com OpenAI: {e}")
        return "Erro: Falha na autenticação com OpenAI.", "OpenAI (GPT-4 Turbo)"
    except Exception as e:
        logging.error(f"Erro ao classificar com OpenAI: {e}")
        return "Erro: OpenAI API Error.", "OpenAI (GPT-4 Turbo)"

def _gemini_api_call(model_name, prompt):
    if not GEMINI_API_KEY:
        raise Exception("Chave da API Gemini não configurada para _gemini_api_call")
    model_gemini = genai.GenerativeModel(model_name)
    return model_gemini.generate_content(prompt)

def classificar_com_gemini(mensagem, exemplos_base):
    if not GEMINI_API_KEY:
        logging.warning("Gemini API Key não configurada.")
        return None, "Gemini (Não Configurado)"
    prompt_completo = construir_prompt_comum(mensagem, exemplos_base)
    try:
        logging.info(f"Enviando para Gemini: '{mensagem[:30]}...'")
        response = api_call_with_retry(_gemini_api_call, 'gemini-pro', prompt_completo)
        if response:
            resultado_ia = response.text.strip().lower()
            if resultado_ia in ["golpe", "não é golpe"]:
                logging.info(f"Gemini classificou como: {resultado_ia}")
                return resultado_ia, "Gemini (Pro)"
            if "golpe" in resultado_ia: return "golpe", "Gemini (Pro)"
            if "não é golpe" in resultado_ia: return "não é golpe", "Gemini (Pro)"
            logging.warning(f"Resposta inesperada do Gemini: '{resultado_ia}'.")
        return None, "Gemini (Pro)"
    except Exception as e:
        logging.error(f"Erro ao classificar com Gemini: {e}")
        if "API_KEY_INVALID" in str(e) or "PERMISSION_DENIED" in str(e) or ("google.api_core.exceptions.PermissionDenied" in str(type(e))):
             return "Erro: Falha na autenticação com Gemini.", "Gemini (Pro)"
        return "Erro: Gemini API Error.", "Gemini (Pro)"


def classificar_com_multi_ia(mensagem):
    exemplos_base = carregar_base()
    classificacao_final = None
    fonte_final = "Nenhuma IA"
    erros_api = []

    if OPENAI_API_KEY:
        resultado_openai, fonte_openai = classificar_com_openai(mensagem, exemplos_base)
        if resultado_openai and "Erro:" not in resultado_openai:
            classificacao_final = resultado_openai
            fonte_final = fonte_openai
        elif resultado_openai and "Erro:" in resultado_openai:
            erros_api.append(f"{fonte_openai}: {resultado_openai.replace('Erro: ', '')}")

    if classificacao_final is None and GEMINI_API_KEY:
        resultado_gemini, fonte_gemini = classificar_com_gemini(mensagem, exemplos_base)
        if resultado_gemini and "Erro:" not in resultado_gemini:
            classificacao_final = resultado_gemini
            fonte_final = fonte_gemini
        elif resultado_gemini and "Erro:" in resultado_gemini:
            erros_api.append(f"{fonte_gemini}: {resultado_gemini.replace('Erro: ', '')}")

    if classificacao_final: # Apenas se houver uma classificação válida (não None e não Erro)
        base_atual = carregar_base()
        base_atualizada = adicionar_nova_entrada(base_atual, mensagem, classificacao_final)
        salvar_base(base_atualizada)
        return f"{classificacao_final} (Fonte: {fonte_final})"
    elif erros_api:
        msg_erro = f"Erro: Falha em serviços de IA ({'; '.join(erros_api)})"
        logging.error(msg_erro)
        return msg_erro
    else:
        msg_incerto = "Classificação incerta (IA). Por favor, tenha cuidado extra com esta mensagem."
        logging.warning(f"IA incerta para: '{mensagem[:30]}...'")
        return msg_incerto


def classificar_mensagem_principal(mensagem):
    global modelo_treinado_global

    if modelo_treinado_global and hasattr(modelo_treinado_global, 'predict_proba'):
        try:
            probabilidades = modelo_treinado_global.predict_proba([mensagem])[0]
            classes = modelo_treinado_global.classes_
            confianca_maxima = np.max(probabilidades)

            if confianca_maxima >= MODELO_LOCAL_CONFIANCA_LIMIAR:
                predicao_idx = np.argmax(probabilidades)
                predicao = classes[predicao_idx]
                if predicao in ["Golpe", "Não é golpe"]:
                    logging.info(f"Classificação local (confiança: {confianca_maxima:.2f}) para '{mensagem[:30]}...': {predicao}")
                    return f"{predicao} (Fonte: Modelo Local, Confiança: {confianca_maxima:.2f})"
                else:
                    logging.warning(f"Modelo local retornou classe inesperada '{predicao}'. Recorrendo à Multi-IA.")
            else:
                logging.info(f"Confiança do modelo local ({confianca_maxima:.2f}) abaixo do limiar ({MODELO_LOCAL_CONFIANCA_LIMIAR}). Recorrendo à Multi-IA.")
        except Exception as e:
            logging.error(f"Erro ao classificar com modelo local ou obter probabilidades: {e}. Recorrendo à Multi-IA.")
    else:
        logging.warning("Modelo local não treinado/carregado ou não suporta predict_proba. Recorrendo à Multi-IA.")

    resultado_ia = classificar_com_multi_ia(mensagem)

    if "Erro:" in resultado_ia and modelo_treinado_global and hasattr(modelo_treinado_global, 'predict'):
        logging.warning(f"Todas as IAs falharam. Usando modelo local como último recurso para '{mensagem[:30]}...'.")
        try:
            predicao_fallback = modelo_treinado_global.predict([mensagem])[0]
            if predicao_fallback in ["Golpe", "Não é golpe"]:
                 return f"{predicao_fallback} (Fonte: Modelo Local - Fallback IA)"
        except Exception as e_fallback:
            logging.error(f"Erro no fallback para modelo local: {e_fallback}")
            return resultado_ia

    return resultado_ia


# ---------- Web Scraping do ReclameAqui ------------
def coletar_reclameaqui(termo="golpe pix", paginas=1):
    base_url = "https://www.reclameaqui.com.br/busca/"
    resultados_scraping = []
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    logging.info(f"Scraping ReclameAqui: '{termo}', {paginas} pág.")
    for pagina in range(1, paginas + 1):
        url = f"{base_url}?query={requests.utils.quote(termo)}&page={pagina}"
        try:
            resp = requests.get(url, headers=headers, timeout=15)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, "html.parser")
            posts = soup.select('a[data-testid="complaint-title"], p[data-testid="complaint-description"]')
            if not posts: break
            for p in posts:
                texto = p.get_text(separator=" ", strip=True)
                if texto and len(texto.split()) > 10 and len(texto) < 1000:
                    resultados_scraping.append(texto)
            time.sleep(API_RETRY_DELAY)
        except requests.exceptions.RequestException as e:
            logging.warning(f"Erro scraping ReclameAqui (pág {pagina}): {e}")
            break
        except Exception as e:
            logging.warning(f"Erro inesperado scraping ReclameAqui (pág {pagina}): {e}")
            break
    logging.info(f"Scraping ReclameAqui concluído. {len(resultados_scraping)} textos.")
    return resultados_scraping

# ---------- Atualização e Retreinamento Automático ------------
def atualizar_e_treinar_automaticamente():
    global modelo_treinado_global
    logging.info("[AGENDADOR] Iniciando atualização e retreinamento.")
    base_existente = carregar_base()
    novos_textos_raspados = coletar_reclameaqui(termo="golpe whatsapp", paginas=1)

    mensagens_realmente_novas = [
        msg for msg in novos_textos_raspados
        if not any(hashlib.sha256(msg.encode("utf-8")).hexdigest() == hashlib.sha256(d["mensagem"].encode("utf-8")).hexdigest() for d in base_existente)
    ]
    logging.info(f"[AGENDADOR] Novas mensagens únicas do ReclameAqui: {len(mensagens_realmente_novas)}.")

    if mensagens_realmente_novas:
        for i, msg_nova in enumerate(mensagens_realmente_novas):
            if i >= 2:
                logging.info(f"[AGENDADOR] Limite de {i} mensagens para IA atingido neste ciclo.")
                break
            logging.info(f"[AGENDADOR] Classificando msg raspada ({i+1}/{len(mensagens_realmente_novas)}) com Multi-IA: '{msg_nova[:30]}...'")
            classificacao_multi_ia = classificar_com_multi_ia(msg_nova)
            if "Erro:" in classificacao_multi_ia:
                logging.warning(f"[AGENDADOR] Falha ao classificar '{msg_nova[:30]}...' com Multi-IA.")

    modelo_atualizado = treinar_modelo()
    if modelo_atualizado:
        modelo_treinado_global = modelo_atualizado
        logging.info("[AGENDADOR] Modelo local re-treinado e atualizado.")
    else:
        logging.warning("[AGENDADOR] Falha ao re-treinar modelo local após coleta.")
    logging.info("[AGENDADOR] Tarefa de atualização e retreinamento concluída.")

# ---------- Interface Flask (Rotas) ------------
@app.route("/", methods=["GET", "POST"])
def home():
    global modelo_treinado_global
    resultado_final_completo = None
    mensagem_submetida = request.form.get("mensagem", "").strip() if request.method == "POST" else request.args.get("mensagem", "")
    classificacao_pura_para_feedback = None

    if request.method == "POST":
        feedback_usuario = request.form.get("feedback")
        classificacao_anterior_pura_form = request.form.get("classificacao_anterior_pura")
        correcao_feedback = request.form.get("correcao_feedback")

        logging.info(f"POST data: mensagem='{mensagem_submetida[:50]}...', feedback='{feedback_usuario}', class_ant_pura='{classificacao_anterior_pura_form}', correcao_feed='{correcao_feedback}'")

        if feedback_usuario and mensagem_submetida and classificacao_anterior_pura_form:
            base_atual = carregar_base()
            classificacao_para_salvar = ""
            if feedback_usuario == "nao" and correcao_feedback:
                classificacao_para_salvar = correcao_feedback
                resultado_final_completo = f"Obrigado pelo feedback! A mensagem foi reclassificada como: {classificacao_para_salvar}."
                logging.info(f"Feedback 'Não' com correção para '{mensagem_submetida[:30]}...'. Anterior: '{classificacao_anterior_pura_form}', Corrigida para: '{classificacao_para_salvar}'.")
            elif feedback_usuario == "sim":
                classificacao_para_salvar = classificacao_anterior_pura_form
                resultado_final_completo = f"Obrigado por confirmar! A classificação '{classificacao_para_salvar}' foi mantida."
                logging.info(f"Feedback 'Sim' para '{mensagem_submetida[:30]}...'. Classificação confirmada: '{classificacao_para_salvar}'.")

            if classificacao_para_salvar:
                logging.info(f"Salvando feedback. Mensagem: '{mensagem_submetida[:30]}', Classificação para salvar: '{classificacao_para_salvar}'")
                base_atualizada = adicionar_nova_entrada(base_atual, mensagem_submetida, classificacao_para_salvar)
                salvar_base(base_atualizada)
                modelo_novo = treinar_modelo()
                if modelo_novo:
                    modelo_treinado_global = modelo_novo
            # Após o feedback, não definimos classificacao_pura_para_feedback para evitar que os botões de feedback apareçam novamente para a mensagem de agradecimento.

        elif mensagem_submetida and not feedback_usuario:
            logging.info(f"Nova mensagem para análise: '{mensagem_submetida[:30]}...'")
            resultado_final_completo = classificar_mensagem_principal(mensagem_submetida)
            logging.info(f"Resultado da classificação para '{mensagem_submetida[:30]}...': {resultado_final_completo}")

            # Define classificacao_pura_para_feedback APENAS se for uma classificação válida
            if resultado_final_completo and isinstance(resultado_final_completo, str) and \
               not any(keyword in resultado_final_completo.lower() for keyword in ["erro:", "classificação incerta", "obrigado pelo", "obrigado por confirmar"]):
                if "não é golpe" in resultado_final_completo.lower():
                    classificacao_pura_para_feedback = "Não é golpe"
                elif "golpe" in resultado_final_completo.lower() : # Garante que "golpe" não está em "não é golpe"
                    classificacao_pura_para_feedback = "Golpe"

    logging.info(f"Para template: resultado_completo='{resultado_final_completo}', msg_submetida='{mensagem_submetida[:30]}...', class_pura_feedback='{classificacao_pura_para_feedback}'")
    return render_template("index.html",
                           resultado_classificacao_completo=resultado_final_completo,
                           mensagem_submetida=mensagem_submetida,
                           classificacao_pura_para_feedback=classificacao_pura_para_feedback)


@app.route("/api/v1/classificar", methods=["POST"])
def api_classificar():
    try:
        dados_requisicao = request.json
        if not dados_requisicao or "mensagem" not in dados_requisicao:
            return jsonify({"erro": "Payload JSON inválido ou 'mensagem' não fornecida."}), 400
        mensagem_api = dados_requisicao.get("mensagem")
        if not isinstance(mensagem_api, str) or not mensagem_api.strip():
            return jsonify({"erro": "'mensagem' deve ser uma string não vazia."}), 400

        resultado_api_completo = classificar_mensagem_principal(mensagem_api)

        classificacao_pura_api = "incerta"
        fonte_api = "desconhecida"
        if resultado_api_completo:
            if "Golpe" in resultado_api_completo and "Não é golpe" not in resultado_api_completo: classificacao_pura_api = "Golpe"
            elif "Não é golpe" in resultado_api_completo: classificacao_pura_api = "Não é golpe"

            if "(Fonte:" in resultado_api_completo:
                try:
                    fonte_api = resultado_api_completo.split("(Fonte:")[1].split(")")[0].strip()
                except: pass

        return jsonify({
            "mensagem": mensagem_api,
            "classificacao": classificacao_pura_api,
            "detalhe_classificacao": resultado_api_completo,
            "fonte_principal": fonte_api
        })
    except Exception as e:
        logging.error(f"Erro no endpoint /api/v1/classificar: {e}")
        return jsonify({"erro": "Erro interno no servidor."}), 500


@app.route("/api/v1/retrain", methods=["POST"])
@require_api_key
def rota_retreinar_modelo():
    global modelo_treinado_global
    logging.info("Requisição autenticada para re-treinar o modelo local via /api/v1/retrain.")
    modelo_novo = treinar_modelo()
    if modelo_novo:
        modelo_treinado_global = modelo_novo
        return jsonify({"mensagem": "Modelo local re-treinado com sucesso!"}), 200
    else:
        return jsonify({"erro": "Falha ao re-treinar o modelo local. Verifique os logs."}), 500

# ---------- Agendador ------------
scheduler = BackgroundScheduler(daemon=True)
try:
    scheduler.add_job(atualizar_e_treinar_automaticamente, 'interval', hours=12)
    scheduler.start()
    logging.info("Agendador iniciado.")
except Exception as e:
    logging.error(f"Erro ao iniciar o agendador: {e}")

# ---------- Função para rodar a aplicação Flask com pyngrok ------------
def try_run_app_with_ngrok(initial_port, max_retries=5):
    port_to_try = initial_port
    ngrok_tunnel = None
    NGROK_TOKEN = os.getenv("NGROK_AUTHTOKEN")
    if NGROK_TOKEN:
        try:
            ngrok.set_auth_token(NGROK_TOKEN)
            logging.info("Token Ngrok configurado.")
        except Exception as e_auth: logging.error(f"Erro config token ngrok: {e_auth}")
    else: logging.warning("NGROK_AUTHTOKEN não encontrado.")

    for i in range(max_retries):
        logging.info(f"Tentando Flask porta {port_to_try} com ngrok (tentativa {i+1}/{max_retries}).")
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.bind(("0.0.0.0", port_to_try))
            s.close()
            logging.info(f"Porta {port_to_try} livre.")
            try:
                for t in ngrok.get_tunnels():
                    ngrok.disconnect(t.public_url)
                    logging.info(f"Túnel ngrok {t.public_url} desconectado.")
                ngrok.kill()
                logging.info("Processo ngrok anterior finalizado.")
            except Exception as ng_clean_e:
                logging.info(f"Limpeza ngrok: {ng_clean_e}")

            ngrok_tunnel = ngrok.connect(port_to_try)
            public_url = ngrok_tunnel.public_url
            logging.info(f"Túnel Ngrok: {public_url}")
            print(f" * Aplicação GolpeGuard: {public_url}")
            app.run(host="0.0.0.0", port=port_to_try, debug=False, use_reloader=False)
            return
        except OSError as e:
            if e.errno == 98: # Address already in use
                logging.warning(f"Porta {port_to_try} em uso.")
                if ngrok_tunnel:
                    try:
                        ngrok.disconnect(ngrok_tunnel.public_url)
                        ngrok.kill()
                    except Exception as ed:
                        logging.error(f"Erro disconnect ngrok: {ed}")
                port_to_try += 1
            else: # Outro OSError
                logging.error(f"Erro SO porta {port_to_try}: {e}")
                if ngrok_tunnel:
                    try:
                        ngrok.disconnect(ngrok_tunnel.public_url)
                        ngrok.kill()
                    except Exception as ed:
                        logging.error(f"Erro disconnect ngrok: {ed}")
                break # Sai do loop de tentativas de porta
        except Exception as e: # Outros erros (incluindo do pyngrok)
            logging.error(f"Erro Flask/ngrok porta {port_to_try}: {e}")
            if "ERR_NGROK_108" in str(e) or "ERR_NGROK_4018" in str(e) or "authentication failed" in str(e).lower():
                print("ERRO AUTENTICAÇÃO NGROK. Verifique NGROK_AUTHTOKEN e conta.")
                print("Certifique-se de que não há outras sessões ngrok ativas na sua conta: https://dashboard.ngrok.com/cloud-edge/agent-sessions")
                logging.error("Falha autenticação Ngrok. Verifique o token e sessões ativas no painel do ngrok.")
            if ngrok_tunnel:
                try:
                    ngrok.disconnect(ngrok_tunnel.public_url)
                    ngrok.kill()
                except Exception as ed:
                    logging.error(f"Erro disconnect ngrok: {ed}")
            # Só tenta outra porta se não for erro de autenticação do ngrok ou limite de sessão
            if not ("ERR_NGROK_108" in str(e) or "ERR_NGROK_4018" in str(e) or "authentication failed" in str(e).lower()):
                 port_to_try += 1
            else: break # Para se for erro de autenticação/limite de sessão
    logging.error(f"Não foi possível iniciar Flask com ngrok após {max_retries} tentativas.")
    print(f"ERRO: Não foi possível iniciar Flask com ngrok.")

if __name__ == "__main__":
    logging.info("Executando atualização e treinamento inicial...")
    try:
        atualizar_e_treinar_automaticamente()
    except Exception as e:
        logging.error(f"Erro na atualização e treinamento inicial: {e}")

    default_port = int(os.getenv("PORT", 5005))
    try_run_app_with_ngrok(initial_port=default_port)


INFO:root:Modelo local carregado de modelo_golpe_guard.pkl.
INFO:apscheduler.scheduler:Adding job tentatively -- it will be properly scheduled when the scheduler starts
INFO:apscheduler.scheduler:Added job "atualizar_e_treinar_automaticamente" to job store "default"
INFO:apscheduler.scheduler:Scheduler started
INFO:root:Agendador iniciado.
INFO:root:Executando atualização e treinamento inicial...
INFO:root:[AGENDADOR] Iniciando atualização e retreinamento.
INFO:root:Scraping ReclameAqui: 'golpe whatsapp', 1 pág.
INFO:root:Scraping ReclameAqui concluído. 0 textos.
INFO:root:[AGENDADOR] Novas mensagens únicas do ReclameAqui: 0.
INFO:root:Modelo local treinado com sucesso.
INFO:root:Modelo local salvo em modelo_golpe_guard.pkl.
INFO:root:Avaliação do modelo local: Acurácia=0.5000
INFO:root:[AGENDADOR] Modelo local re-treinado e atualizado.
INFO:root:[AGENDADOR] Tarefa de atualização e retreinamento concluída.
INFO:root:Tentando Flask porta 5005 com ngrok (tentativa 1/5).
INFO:root:Porta 5

 * Aplicação GolpeGuard: https://3e36-35-245-103-146.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5005
 * Running on http://172.28.0.12:5005
INFO:werkzeug:Press CTRL+C to quit
INFO:pyngrok.process.ngrok:t=2025-05-17T03:26:10+0000 lvl=info msg="join connections" obj=join id=b439cf1d8ad5 l=127.0.0.1:5005 r=[2804:29b8:5078:9975:19d4:8ac6:13d0:f9c2]:59780
INFO:root:Para template: resultado_completo='None', msg_submetida='...', class_pura_feedback='None'
INFO:werkzeug:127.0.0.1 - - [17/May/2025 03:26:10] "GET / HTTP/1.1" 200 -
INFO:pyngrok.process.ngrok:t=2025-05-17T03:26:11+0000 lvl=info msg="join connections" obj=join id=d8e090f73154 l=127.0.0.1:5005 r=[2804:29b8:5078:9975:19d4:8ac6:13d0:f9c2]:59780
INFO:werkzeug:127.0.0.1 - - [17/May/2025 03:26:11] "GET /favicon.ico HTTP/1.1" 404 -
INFO:pyngrok.process.ngrok:t=2025-05-17T03:26:34+0000 lvl=info msg="join connections" obj=join id=1f1e17a

In [ ]:
# Célula para Salvar o templates/index.html no Google Colab

import os

# Cria o diretório 'templates' se não existir
# Esta linha deve estar corretamente indentada (sem espaços à esquerda,
# se for o início de um bloco de código na célula)
if not os.path.exists("templates"):
    os.makedirs("templates")
    print("Diretório 'templates' criado.")

# O conteúdo HTML é atribuído a esta variável.
# A linha html_content = """ deve estar corretamente indentada.
html_content = """
<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>GolpeGuard Turbo Inteligente</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <style>
        body {
            font-family: 'Inter', sans-serif;
        }
        .loader {
            border: 4px solid #f3f3f3; /* Light grey */
            border-top: 4px solid #3498db; /* Blue */
            border-radius: 50%;
            width: 30px;
            height: 30px;
            animation: spin 1s linear infinite;
            margin: 1rem auto;
        }
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
    </style>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap" rel="stylesheet">
</head>
<body class="bg-gradient-to-br from-slate-900 to-slate-700 text-gray-100 min-h-screen flex flex-col items-center justify-center p-4">

    <div class="bg-slate-800 shadow-2xl rounded-xl p-6 md:p-10 w-full max-w-2xl">
        <header class="text-center mb-8">
            <h1 class="text-4xl font-bold text-sky-400 mb-2">
                <span role="img" aria-label="Escudo de Proteção" class="mr-2">🛡️</span>GolpeGuard TI
            </h1>
            <p class="text-lg text-slate-300">Seu detector de golpes inteligente, agora com Turbo!</p>
        </header>

        <form method="POST" id="analysis-form" class="space-y-6">
            <div>
                <label for="mensagem" class="block text-sm font-medium text-sky-300 mb-1">Mensagem para Análise:</label>
                <textarea name="mensagem" id="mensagem" rows="6"
                          class="w-full p-3 bg-slate-700 border border-slate-600 rounded-lg shadow-sm focus:ring-2 focus:ring-sky-500 focus:border-sky-500 placeholder-slate-400 text-gray-100 transition duration-150 ease-in-out"
                          placeholder="Cole aqui a mensagem suspeita que você recebeu por SMS, WhatsApp, e-mail, etc...">{{ mensagem_submetida if mensagem_submetida else '' }}</textarea>
            </div>

            <div class="text-center">
                <button type="submit" id="submit-button"
                        class="w-full sm:w-auto bg-sky-600 hover:bg-sky-700 text-white font-semibold py-3 px-6 rounded-lg shadow-md focus:outline-none focus:ring-2 focus:ring-sky-500 focus:ring-opacity-50 transition duration-150 ease-in-out transform hover:scale-105">
                    Analisar Mensagem
                </button>
            </div>
        </form>

        <div id="loader" class="loader" style="display: none;"></div>

        {% if resultado_classificacao_completo %}
            <div id="resultado-bloco" class="mt-8 p-6 rounded-lg shadow-lg border
                {% if 'Golpe' in resultado_classificacao_completo and 'Não é golpe' not in resultado_classificacao_completo %} bg-red-800 border-red-600
                {% elif 'Não é golpe' in resultado_classificacao_completo %} bg-green-800 border-green-600
                {% elif 'Obrigado' in resultado_classificacao_completo %} bg-sky-800 border-sky-600
                {% else %} bg-yellow-800 border-yellow-600
                {% endif %}">

                <h2 class="text-2xl font-semibold mb-3 text-center">
                    {% if 'Golpe' in resultado_classificacao_completo and 'Não é golpe' not in resultado_classificacao_completo %} 🚨 Potencial Golpe!
                    {% elif 'Não é golpe' in resultado_classificacao_completo %} ✅ Parece Seguro.
                    {% elif 'Obrigado' in resultado_classificacao_completo %} ℹ️ Status
                    {% else %} ⚠️ Atenção!
                    {% endif %}
                </h2>
                <p class="text-lg text-center mb-4">{{ resultado_classificacao_completo }}</p>

                {% if classificacao_pura_para_feedback and not 'Obrigado' in resultado_classificacao_completo %}
                    <div class="mt-6 border-t border-slate-600 pt-4">
                        <p class="text-center text-sm text-slate-300 mb-2">A classificação está correta?</p>
                        <form method="POST" id="feedback-form" class="space-y-3">
                            <input type="hidden" name="mensagem" value="{{ mensagem_submetida }}">
                            <input type="hidden" name="classificacao_anterior_pura" value="{{ classificacao_pura_para_feedback }}">

                            <div class="flex justify-center items-center space-x-3">
                                <button name="feedback" value="sim" type="submit"
                                        class="bg-green-600 hover:bg-green-700 text-white font-medium py-2 px-4 rounded-md text-sm transition">
                                    👍 Sim
                                </button>
                                <button name="feedback" value="nao" type="button" id="feedback-nao-btn"
                                        class="bg-red-600 hover:bg-red-700 text-white font-medium py-2 px-4 rounded-md text-sm transition">
                                    👎 Não
                                </button>
                            </div>
                            <div id="correcao-feedback-div" class="mt-3 text-center feedback-options" style="display: none;">
                                <p class="text-sm text-slate-300 mb-1">Qual seria a classificação correta?</p>
                                <label><input type="radio" name="correcao_feedback" value="Golpe" required> Golpe</label>
                                <label><input type="radio" name="correcao_feedback" value="Não é golpe"> Não é golpe</label>
                                <button type="submit" id="submit-correcao-btn" class="mt-2 bg-sky-600 hover:bg-sky-700 text-white font-medium py-1 px-3 rounded-md text-sm">Enviar Correção</button>
                            </div>
                        </form>
                    </div>
                {% endif %}
            </div>
        {% endif %}

        <footer class="mt-10 text-center text-xs text-slate-400">
            <p>&copy; 2024 GolpeGuard TI. Todos os direitos reservados.</p> <p class="mt-1">Lembre-se: este é um sistema de auxílio. Sempre use o bom senso.</p>
        </footer>
    </div>

    <script>
        const analysisForm = document.getElementById('analysis-form');
        const submitButton = document.getElementById('submit-button');
        const loader = document.getElementById('loader');
        const resultadoBloco = document.getElementById('resultado-bloco');
        const feedbackNaoBtn = document.getElementById('feedback-nao-btn');
        const correcaoFeedbackDiv = document.getElementById('correcao-feedback-div');
        const feedbackForm = document.getElementById('feedback-form');

        if (analysisForm) {
            analysisForm.addEventListener('submit', function(event) {
                const mensagemTextarea = document.getElementById('mensagem');
                if (mensagemTextarea && mensagemTextarea.value.trim() !== '') {
                    if (submitButton) {
                        submitButton.disabled = true;
                        submitButton.innerHTML = 'Analisando... <span class="animate-pulse">⏳</span>';
                    }
                    if (loader) loader.style.display = 'block';
                    if (resultadoBloco) resultadoBloco.style.display = 'none';
                } else {
                    event.preventDefault();
                }
            });
        }

        if (feedbackNaoBtn) {
            feedbackNaoBtn.addEventListener('click', function() {
                if (correcaoFeedbackDiv) correcaoFeedbackDiv.style.display = 'block';
                let feedbackInput = feedbackForm.querySelector('input[name="feedback"]');
                if (!feedbackInput) {
                    feedbackInput = document.createElement('input');
                    feedbackInput.type = 'hidden';
                    feedbackInput.name = 'feedback';
                    feedbackForm.appendChild(feedbackInput);
                }
                feedbackInput.value = 'nao';
            });
        }

        window.addEventListener('DOMContentLoaded', () => {
            const resultadoBlocoPresente = document.getElementById('resultado-bloco');
            const loaderElement = document.getElementById('loader');
            if (resultadoBlocoPresente) { // Se o Jinja renderizou o bloco de resultado
                resultadoBlocoPresente.style.display = 'block';
                if (loaderElement) loaderElement.style.display = 'none';
                if (submitButton) {
                    submitButton.disabled = false;
                    submitButton.innerHTML = 'Analisar Mensagem';
                }
            }
        });
    </script>
</body>
</html>
"""

# As linhas with open(...) e print(...) devem estar corretamente indentadas
# em relação ao início da célula ou ao bloco if anterior.
# Se não houver um 'if' antes, elas devem estar sem espaços à esquerda.
with open("templates/index.html", "w", encoding="utf-8") as f:
    f.write(html_content)
print("Arquivo templates/index.html salvo.")


In [ ]:
<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>GolpeGuard Turbo Inteligente</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <style>
        body {
            font-family: 'Inter', sans-serif;
        }
        .loader {
            border: 4px solid #f3f3f3; /* Light grey */
            border-top: 4px solid #3498db; /* Blue */
            border-radius: 50%;
            width: 30px;
            height: 30px;
            animation: spin 1s linear infinite;
            margin: 1rem auto;
        }
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
    </style>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap" rel="stylesheet">
</head>
<body class="bg-gradient-to-br from-slate-900 to-slate-700 text-gray-100 min-h-screen flex flex-col items-center justify-center p-4">

    <div class="bg-slate-800 shadow-2xl rounded-xl p-6 md:p-10 w-full max-w-2xl">
        <header class="text-center mb-8">
            <h1 class="text-4xl font-bold text-sky-400 mb-2">
                <span role="img" aria-label="Escudo de Proteção" class="mr-2">🛡️</span>GolpeGuard TI
            </h1>
            <p class="text-lg text-slate-300">Seu detector de golpes inteligente, agora com Turbo!</p>
        </header>

        <form method="POST" id="analysis-form" class="space-y-6">
            <div>
                <label for="mensagem" class="block text-sm font-medium text-sky-300 mb-1">Mensagem para Análise:</label>
                <textarea name="mensagem" id="mensagem" rows="6"
                          class="w-full p-3 bg-slate-700 border border-slate-600 rounded-lg shadow-sm focus:ring-2 focus:ring-sky-500 focus:border-sky-500 placeholder-slate-400 text-gray-100 transition duration-150 ease-in-out"
                          placeholder="Cole aqui a mensagem suspeita que você recebeu por SMS, WhatsApp, e-mail, etc...">{{ mensagem_submetida if mensagem_submetida else '' }}</textarea>
            </div>

            <div class="text-center">
                <button type="submit" id="submit-button"
                        class="w-full sm:w-auto bg-sky-600 hover:bg-sky-700 text-white font-semibold py-3 px-6 rounded-lg shadow-md focus:outline-none focus:ring-2 focus:ring-sky-500 focus:ring-opacity-50 transition duration-150 ease-in-out transform hover:scale-105">
                    Analisar Mensagem
                </button>
            </div>
        </form>

        <div id="loader" class="loader" style="display: none;"></div>

        {% if resultado_classificacao %}
            <div id="resultado-bloco" class="mt-8 p-6 rounded-lg shadow-lg
                {% if 'Golpe' in resultado_classificacao %} bg-red-700 border-red-500
                {% elif 'Não é golpe' in resultado_classificacao %} bg-green-700 border-green-500
                {% elif 'Obrigado pelo feedback' in resultado_classificacao %} bg-sky-700 border-sky-500
                {% else %} bg-yellow-700 border-yellow-500
                {% endif %} border" style="display: block;"> {/* Adicionado style="display: block;" para garantir visibilidade se renderizado */}

                <h2 class="text-2xl font-semibold mb-3 text-center">
                    {% if 'Golpe' in resultado_classificacao %}
                        <span role="img" aria-label="Alerta Vermelho">🚨</span> Potencial Golpe Detectado!
                    {% elif 'Não é golpe' in resultado_classificacao %}
                        <span role="img" aria-label="Verificado">✅</span> Parece Seguro.
                    {% elif 'Obrigado pelo feedback' in resultado_classificacao %}
                        <span role="img" aria-label="Informação">ℹ️</span> Status
                    {% else %}
                        <span role="img" aria-label="Atenção">⚠️</span> Atenção!
                    {% endif %}
                </h2>
                <p class="text-lg text-center mb-4">{{ resultado_classificacao }}</p>

                {% if classificacao_para_feedback and not 'Obrigado pelo feedback' in resultado_classificacao %}
                    <div class="mt-6 border-t border-slate-500 pt-4">
                        <p class="text-center text-sm text-slate-300 mb-2">A classificação está correta?</p>
                        <form method="POST" class="flex justify-center items-center space-x-3">
                            <input type="hidden" name="mensagem" value="{{ mensagem_submetida }}">
                            <input type="hidden" name="classificacao_anterior" value="{{ classificacao_para_feedback }}">

                            <button name="feedback" value="sim" type="submit"
                                    class="bg-green-600 hover:bg-green-700 text-white font-medium py-2 px-4 rounded-md text-sm transition duration-150 ease-in-out">
                                👍 Sim
                            </button>
                            <button name="feedback" value="nao" type="submit"
                                    class="bg-red-600 hover:bg-red-700 text-white font-medium py-2 px-4 rounded-md text-sm transition duration-150 ease-in-out">
                                👎 Não
                            </button>
                        </form>
                    </div>
                {% endif %}
            </div>
        {% endif %}

        <footer class="mt-10 text-center text-xs text-slate-400">
            <p>&copy; 2024 GolpeGuard Turbo Inteligente. Todos os direitos reservados.</p>
            <p class="mt-1">Lembre-se: este é um sistema de auxílio. Sempre use o bom senso.</p>
        </footer>
    </div>

    <script>
        const analysisForm = document.getElementById('analysis-form');
        const submitButton = document.getElementById('submit-button');
        const loader = document.getElementById('loader');
        const resultadoBloco = document.getElementById('resultado-bloco');

        if (analysisForm) {
            analysisForm.addEventListener('submit', function(event) {
                const mensagemTextarea = document.getElementById('mensagem');
                if (mensagemTextarea && mensagemTextarea.value.trim() !== '') {
                    if (submitButton) {
                        submitButton.disabled = true;
                        submitButton.innerHTML = 'Analisando... <span class="animate-pulse">⏳</span>';
                    }
                    if (loader) loader.style.display = 'block';
                    if (resultadoBloco) resultadoBloco.style.display = 'none';
                } else {
                    // Impede o envio do formulário se a mensagem estiver vazia,
                    // embora o 'required' no textarea já deva fazer isso.
                    event.preventDefault();
                }
            });
        }

        // Garante que o bloco de resultado seja exibido se ele contiver conteúdo ao carregar a página.
        // E esconde o loader se o resultado já estiver presente.
        window.addEventListener('DOMContentLoaded', () => {
            const resultadoBlocoPresente = document.getElementById('resultado-bloco');
            const loaderElement = document.getElementById('loader');

            // Verifica se o bloco de resultado existe E se ele tem conteúdo visível (não apenas espaços em branco)
            // A verificação `offsetParent !== null` é uma forma de checar se o elemento está de fato visível no layout.
            // No entanto, como estamos a definir `style="display: block;"` no HTML se o Jinja o renderizar,
            // podemos simplificar a verificação para apenas se o elemento existe.
            if (resultadoBlocoPresente) {
                resultadoBlocoPresente.style.display = 'block'; // Garante que está visível
                if (loaderElement) loaderElement.style.display = 'none';

                // Reabilita o botão se o resultado for exibido (após o recarregamento da página)
                if (submitButton) {
                    submitButton.disabled = false;
                    submitButton.innerHTML = 'Analisar Mensagem';
                }
            }
        });
    </script>
</body>
</html>
